In [1]:
import pandas as pd
df=pd.read_csv('data/products_reviews.csv')[['index','product/productId','review/userId','review/time','review/helpfulness','review/score']]
df.columns=['index','product_id','user_id','time','helpfulness','score']
def help(x):
    a,b=int(x.split('/')[0]),int(x.split('/')[1])
    if b==0: return 0 
    else: return a/b
df['helpfulness']=df['helpfulness'].apply(lambda x:help(x))
df.head(10)
df.shape[0]

200707

In [7]:
df.groupby(['user_id'])['index'].nunique().reset_index()['index'].describe()

count    165533.000000
mean          1.212489
std           1.275035
min           1.000000
25%           1.000000
50%           1.000000
75%           1.000000
max         329.000000
Name: index, dtype: float64

In [8]:
df.groupby(['product_id'])['index'].nunique().reset_index()['index'].describe()

count    37512.000000
mean         5.350475
std         20.292343
min          1.000000
25%          1.000000
50%          1.000000
75%          3.000000
max       1791.000000
Name: index, dtype: float64

categories

In [20]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
with open('data/categories.txt', 'r', encoding='ISO-8859-1') as file:
    data = file.read().splitlines()
ids=[i for i in data if i[0]!=' ']
cates=[i.strip() for i in data if i[0]==' ']
ids=[i for i in data if i[0]!=' ']
cates=[i.strip() for i in data if i[0]==' ']
k=[i for i in range(len(data)) if data[i][0]!=' ']+[len(data)]
k=[k[i+1]-k[i]-1 for i in range(len(k)-1)]
productId=[]
for i in range(len(ids)):productId+=[ids[i]]*k[i]
cate=pd.DataFrame()
cate['productId']=productId
cate['cate']=[i.split(',')[0] for i in cates]
cate['cate'] = le.fit_transform(cate['cate'])
cate=cate.drop_duplicates(subset=['productId'])
df=pd.merge(df,cate.rename({'productId':'product_id'},axis=1),how='left',on=['product_id'])
df.shape[0]

200707

train test split

In [21]:
import numpy as np
np.random.seed(42)
train = df.sample(frac=0.7, random_state=7)
test = df.drop(train.index.tolist())
train.shape[0],test.shape[0]

(140495, 60212)

In [22]:
train.to_csv('train.csv',index=False)
test.to_csv('test.csv',index=False)

In [23]:
train.head()

,index,product_id,user_id,time,helpfulness,score,cate
145565,145565,B000DZLWHW,A3DP6SK80M7RYJ,2010/09/18,0.0,5.0,46
63962,63962,B000FNGPB0,ABNEHK5WEUYFM,2010/11/16,0.0,5.0,37
33012,33012,B000MEAA22,A39AG4GNGGDGJ9,2009/08/17,0.0,3.0,15
44561,44561,B000NVH0H2,A12WEG8BNXM3HT,2010/08/26,0.0,5.0,37
92663,92663,B0007KPI2S,A13O9OJW8W2MF0,2010/04/19,0.0,2.0,46


Get features

In [31]:
#id label encoding
import pandas as pd
from sklearn.preprocessing import LabelEncoder
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
df=pd.concat([train,test])
le_u = LabelEncoder()
us=le_u.fit_transform(df['user_id'])
train['user_id']=us[:train.shape[0]]
test['user_id']=us[train.shape[0]:]
le_i = LabelEncoder()
it=le_u.fit_transform(df['product_id'])
train['product_id']=it[:train.shape[0]]
test['product_id']=it[train.shape[0]:]
train.head(10)

,index,product_id,user_id,time,helpfulness,score,cate
0,145565,20310,104238,2010/09/18,0.0,5.0,46
1,63962,23917,135920,2010/11/16,0.0,5.0,37
2,33012,31974,98890,2009/08/17,0.0,3.0,15
3,44561,33923,3566,2010/08/26,0.0,5.0,37
4,92663,13685,4502,2010/04/19,0.0,2.0,46
5,18374,34322,40989,2008/05/12,0.0,2.0,15
6,57418,11048,90787,2011/03/10,0.0,5.0,37
7,150604,7802,99969,2013/01/04,0.0,5.0,46
8,41342,1271,131717,2009/12/30,1.0,5.0,37
9,19129,22837,90494,2011/01/16,0.0,4.0,15


In [32]:
uf=train.groupby(['user_id'])[['score','helpfulness']].mean().reset_index().rename({'score':'user_mean_score','helpfulness':'user_mean_help'},axis=1)
pf=train.groupby(['product_id'])[['score','helpfulness']].mean().reset_index().rename({'score':'pro_mean_score','helpfulness':'pro_mean_help'},axis=1)
pf_c=train.groupby(['cate'])[['score','helpfulness']].mean().reset_index().rename({'score':'cate_mean_score','helpfulness':'cate_mean_help'},axis=1)

def mg(data):
    data = pd.merge(pd.merge(pd.merge(data, uf, how='left', on='user_id'), pf, how='left', on='product_id'), pf_c, how='left', on='cate')
    for i in ['score','help']:
        data[f'user_mean_{i}']=data[f'user_mean_{i}'].fillna(uf[f'user_mean_{i}'].mean())
        data[f'pro_mean_{i}'] = data[f'pro_mean_{i}'].fillna(data[f'cate_mean_{i}'])
        data[f'pro_mean_{i}'] = data[f'pro_mean_{i}'].fillna(pf[f'pro_mean_{i}'].mean())
        data=data.drop(f'cate_mean_{i}',axis=1)
    return data
train=mg(train)
test=mg(test)
print(train.shape,test.shape)
train.head(10)


(140495, 11) (60212, 11)


,index,product_id,user_id,time,helpfulness,score,cate,user_mean_score,user_mean_help,pro_mean_score,pro_mean_help
0,145565,20310,104238,2010/09/18,0.0,5.0,46,5.0,0.0,5.000000,0.000000
1,63962,23917,135920,2010/11/16,0.0,5.0,37,5.0,0.0,3.887324,0.278946
2,33012,31974,98890,2009/08/17,0.0,3.0,15,3.0,0.0,2.818182,0.363636
3,44561,33923,3566,2010/08/26,0.0,5.0,37,5.0,0.0,4.285714,0.000000
4,92663,13685,4502,2010/04/19,0.0,2.0,46,2.0,0.0,2.923077,0.497894
5,18374,34322,40989,2008/05/12,0.0,2.0,15,2.0,0.0,3.652174,0.130435
6,57418,11048,90787,2011/03/10,0.0,5.0,37,5.0,0.0,4.166667,0.432099
7,150604,7802,99969,2013/01/04,0.0,5.0,46,5.0,0.0,4.721154,0.184726
8,41342,1271,131717,2009/12/30,1.0,5.0,37,5.0,1.0,5.000000,1.000000
9,19129,22837,90494,2011/01/16,0.0,4.0,15,4.0,0.0,3.411765,0.137255


Review text (smoothing)

In [33]:
import pickle 
import pandas as pd
vecs=pickle.load(open('data/vecs_fc_64_1.pkl','rb'))
vecs.shape

(140495, 64)

In [34]:
train['time'] = pd.to_datetime(train['time'], format='%Y/%m/%d').apply(lambda x: x.timestamp())
test['time'] = pd.to_datetime(test['time'], format='%Y/%m/%d').apply(lambda x: x.timestamp())
train=train.reset_index().rename({'level_0':'idx'},axis=1).sort_values(by=['time'])
train.head(10)

,idx,index,product_id,user_id,time,helpfulness,score,cate,user_mean_score,user_mean_help,pro_mean_score,pro_mean_help
69919,69919,176270,29873,76035,1.200010e+09,0.666667,4.0,57,4.0,0.666667,4.231707,0.274882
64716,64716,34371,26252,71133,1.200010e+09,1.000000,2.0,15,3.5,0.500000,3.230769,0.461538
95170,95170,168595,18051,96666,1.200010e+09,1.000000,5.0,61,5.0,1.000000,4.375000,0.500000
109872,109872,94854,29346,99204,1.200010e+09,0.857143,4.0,46,4.0,0.857143,4.400000,0.737733
119840,119840,195123,17331,10901,1.200010e+09,0.000000,5.0,61,5.0,0.500000,5.000000,0.000000
49031,49031,157375,12024,111061,1.200010e+09,1.000000,1.0,57,1.0,1.000000,1.000000,1.000000
63362,63362,48204,26807,7802,1.200010e+09,1.000000,1.0,37,1.0,1.000000,3.250000,0.500000
17755,17755,64525,34376,30096,1.200010e+09,0.947368,1.0,37,1.0,0.947368,2.545455,0.358852
66143,66143,41219,2716,98701,1.200010e+09,0.000000,4.0,37,4.0,0.000000,4.333333,0.000000
34847,34847,17303,14962,15719,1.200010e+09,0.000000,5.0,15,5.0,0.000000,4.000000,0.200000


In [35]:
import numpy as np
from tqdm import tqdm,trange
def exponential_smoothing(matrix, alpha=0.6):
    smoothed=matrix[0]
    for i in range(1, len(matrix)):
        smoothed = alpha * matrix[i] + (1 - alpha) * smoothed
    return smoothed

us_train=list(train['user_id'].unique())
ps_train=list(train['product_id'].unique())
print(len(us_train),len(ps_train))
mp_u,mp_p={},{}
for u in tqdm(us_train):
    idx=np.array(train[train['user_id']==u]['idx'])
    mp_u[u]=exponential_smoothing(vecs[idx])
for p in tqdm(ps_train):
    idx=np.array(train[train['product_id']==p]['idx'])
    mp_p[p]=exponential_smoothing(vecs[idx])
len(mp_u),len(mp_p)

120471 31092


  0%|          | 0/120471 [00:00<?, ?it/s]

100%|██████████| 31092/31092 [00:14<00:00, 2148.33it/s]


(120471, 31092)

In [36]:
u_mean=np.vstack(list(mp_u.values())).mean(axis=0)
p_mean=np.vstack(list(mp_p.values())).mean(axis=0)
cate_mean={}
for c in tqdm(train['cate'].unique()):
    tmp=[mp_p[i] for i in train[train['cate']==c]['product_id'].unique()]
    cate_mean[c]=np.vstack(tmp).mean(axis=0)
u_mean.shape,p_mean.shape,len(cate_mean)


100%|██████████| 16/16 [00:00<00:00, 172.05it/s]


((64,), (64,), 16)

In [37]:
def get_vecs(data):
    u_vecs,p_vecs,dots=[],[],[]
    for _,r in data.iterrows():
        if r['user_id'] in mp_u: u_vec=mp_u[r['user_id']]
        else: u_vec=u_mean
        if r['product_id'] in mp_p: p_vec=mp_p[r['product_id']] 
        elif r['cate'] in cate_mean: p_vec=cate_mean[r['cate']] 
        else: p_vec=p_mean
        u_vec,p_vec=u_vec/np.linalg.norm(u_vec),p_vec/np.linalg.norm(p_vec)
        u_vecs.append(u_vec)
        p_vecs.append(p_vec)
        dots.append(u_vec@p_vec)
    return u_vecs,p_vecs,dots

train['u_vec'],train['p_vec'],train['dot']=get_vecs(train)
test['u_vec'],test['p_vec'],test['dot']=get_vecs(test)
train.head()

,idx,index,product_id,user_id,time,helpfulness,score,cate,user_mean_score,user_mean_help,pro_mean_score,pro_mean_help,u_vec,p_vec,dot
69919,69919,176270,29873,76035,1.200010e+09,0.666667,4.0,57,4.0,0.666667,4.231707,0.274882,"[-0.023424821, -0.11292456, 0.18853657, -0.115...","[-0.021083262, -0.1304503, 0.16853152, -0.1036...",0.996360
64716,64716,34371,26252,71133,1.200010e+09,1.000000,2.0,15,3.5,0.500000,3.230769,0.461538,"[-0.011791757, -0.12723447, 0.19309308, -0.109...","[0.05222546, -0.12136055, 0.20574287, -0.11758...",0.976426
95170,95170,168595,18051,96666,1.200010e+09,1.000000,5.0,61,5.0,1.000000,4.375000,0.500000,"[-0.02140619, -0.13102354, 0.16955456, -0.1483...","[0.00080324546, -0.12977651, 0.18457516, -0.10...",0.957980
109872,109872,94854,29346,99204,1.200010e+09,0.857143,4.0,46,4.0,0.857143,4.400000,0.737733,"[-0.03225298, -0.11813351, 0.17550687, -0.1327...","[-0.0184484, -0.13732621, 0.1701287, -0.102108...",0.990932
119840,119840,195123,17331,10901,1.200010e+09,0.000000,5.0,61,5.0,0.500000,5.000000,0.000000,"[-0.027470699, -0.11355752, 0.17480963, -0.135...","[-0.032984387, -0.12086839, 0.17090109, -0.136...",0.998860


In [38]:
test.head()

,index,product_id,user_id,time,helpfulness,score,cate,user_mean_score,user_mean_help,pro_mean_score,pro_mean_help,u_vec,p_vec,dot
0,0,30088,34501,1.233360e+09,0.0,1.0,15,4.053897,0.331154,3.589192,0.247138,"[-0.0076882686, -0.1261441, 0.18335178, -0.115...","[0.00032921784, -0.12477857, 0.18587488, -0.10...",0.998717
1,4,17681,33728,1.212365e+09,1.0,5.0,15,4.053897,0.331154,4.000000,0.000000,"[-0.0076882686, -0.1261441, 0.18335178, -0.115...","[0.01309923, -0.13805045, 0.20096067, -0.09707...",0.986891
2,10,69,27202,1.353974e+09,1.0,4.0,15,4.053897,0.331154,3.552239,0.403874,"[-0.0076882686, -0.1261441, 0.18335178, -0.115...","[0.0025823559, -0.12812333, 0.17530991, -0.113...",0.998774
3,11,69,82543,1.350518e+09,1.0,1.0,15,4.053897,0.331154,3.552239,0.403874,"[-0.0076882686, -0.1261441, 0.18335178, -0.115...","[0.0025823559, -0.12812333, 0.17530991, -0.113...",0.998774
4,14,69,18135,1.339200e+09,1.0,3.0,15,4.053897,0.331154,3.552239,0.403874,"[-0.0076882686, -0.1261441, 0.18335178, -0.115...","[0.0025823559, -0.12812333, 0.17530991, -0.113...",0.998774


In [39]:
train['dot'].describe()

count    140495.000000
mean          0.989628
std           0.018902
min          -0.175809
25%           0.987900
50%           0.994004
75%           0.997333
max           1.000000
Name: dot, dtype: float64

In [40]:
test['dot'].describe()

count    60212.000000
mean         0.993290
std          0.012404
min          0.337276
25%          0.992581
50%          0.996365
75%          0.997976
max          1.000000
Name: dot, dtype: float64

In [41]:
import pickle 
pickle.dump(train,open('train1_1.pkl','wb'))
pickle.dump(test,open('test1_1.pkl','wb'))

In [42]:
import pickle 
import pandas as pd
vecs=pickle.load(open('data/vecs_pca_64_1.pkl','rb'))
vecs.shape

(140495, 64)

In [43]:
us_train=list(train['user_id'].unique())
ps_train=list(train['product_id'].unique())
print(len(us_train),len(ps_train))
mp_u,mp_p={},{}
for u in tqdm(us_train):
    idx=np.array(train[train['user_id']==u]['idx'])
    mp_u[u]=exponential_smoothing(vecs[idx])
for p in tqdm(ps_train):
    idx=np.array(train[train['product_id']==p]['idx'])
    mp_p[p]=exponential_smoothing(vecs[idx])
print(len(mp_u),len(mp_p))

u_mean=np.vstack(list(mp_u.values())).mean(axis=0)
p_mean=np.vstack(list(mp_p.values())).mean(axis=0)
cate_mean={}
for c in tqdm(train['cate'].unique()):
    tmp=[mp_p[i] for i in train[train['cate']==c]['product_id'].unique()]
    cate_mean[c]=np.vstack(tmp).mean(axis=0)
print(u_mean.shape,p_mean.shape,len(cate_mean))

def get_vecs(data):
    u_vecs,p_vecs,dots=[],[],[]
    for _,r in data.iterrows():
        if r['user_id'] in mp_u: u_vec=mp_u[r['user_id']]
        else: u_vec=u_mean
        if r['product_id'] in mp_p: p_vec=mp_p[r['product_id']] 
        elif r['cate'] in cate_mean: p_vec=cate_mean[r['cate']] 
        else: p_vec=p_mean
        u_vec,p_vec=u_vec/np.linalg.norm(u_vec),p_vec/np.linalg.norm(p_vec)
        u_vecs.append(u_vec)
        p_vecs.append(p_vec)
        dots.append(u_vec@p_vec)
    return u_vecs,p_vecs,dots

train['u_vec_pca'],train['p_vec_pca'],train['dot_pca']=get_vecs(train)
test['u_vec_pca'],test['p_vec_pca'],test['dot_pca']=get_vecs(test)
print(train.shape,test.shape)
train.head()


120471 31092


100%|██████████| 31092/31092 [00:25<00:00, 1220.24it/s]


120471 31092


100%|██████████| 16/16 [00:00<00:00, 111.75it/s]


(64,) (64,) 16
(140495, 18) (60212, 17)


,idx,index,product_id,user_id,time,helpfulness,score,cate,user_mean_score,user_mean_help,pro_mean_score,pro_mean_help,u_vec,p_vec,dot,u_vec_pca,p_vec_pca,dot_pca
69919,69919,176270,29873,76035,1.200010e+09,0.666667,4.0,57,4.0,0.666667,4.231707,0.274882,"[-0.023424821, -0.11292456, 0.18853657, -0.115...","[-0.021083262, -0.1304503, 0.16853152, -0.1036...",0.996360,"[-0.71635073, 0.12255365, -0.21689054, 0.19962...","[-0.7903225, -0.21802181, 0.05431329, -0.09516...",0.513905
64716,64716,34371,26252,71133,1.200010e+09,1.000000,2.0,15,3.5,0.500000,3.230769,0.461538,"[-0.011791757, -0.12723447, 0.19309308, -0.109...","[0.05222546, -0.12136055, 0.20574287, -0.11758...",0.976426,"[-0.43207568, -0.28160614, -0.37597206, -0.220...","[0.67142487, -0.24425057, -0.3592007, -0.22807...",-0.062924
95170,95170,168595,18051,96666,1.200010e+09,1.000000,5.0,61,5.0,1.000000,4.375000,0.500000,"[-0.02140619, -0.13102354, 0.16955456, -0.1483...","[0.00080324546, -0.12977651, 0.18457516, -0.10...",0.957980,"[-0.23511194, -0.36231145, -0.05822904, 0.3099...","[-0.40747398, 0.28078416, 0.34387022, 0.152414...",0.061640
109872,109872,94854,29346,99204,1.200010e+09,0.857143,4.0,46,4.0,0.857143,4.400000,0.737733,"[-0.03225298, -0.11813351, 0.17550687, -0.1327...","[-0.0184484, -0.13732621, 0.1701287, -0.102108...",0.990932,"[0.5198544, -0.117018275, 0.07950021, 0.314501...","[-0.35075575, 0.14392184, 0.22873686, 0.303211...",0.077728
119840,119840,195123,17331,10901,1.200010e+09,0.000000,5.0,61,5.0,0.500000,5.000000,0.000000,"[-0.027470699, -0.11355752, 0.17480963, -0.135...","[-0.032984387, -0.12086839, 0.17090109, -0.136...",0.998860,"[0.4537118, 0.031590253, 0.70554, -0.15370068,...","[0.51794595, 0.0035934413, 0.5631712, -0.26780...",0.809477


In [44]:
test.head()

,index,product_id,user_id,time,helpfulness,score,cate,user_mean_score,user_mean_help,pro_mean_score,pro_mean_help,u_vec,p_vec,dot,u_vec_pca,p_vec_pca,dot_pca
0,0,30088,34501,1.233360e+09,0.0,1.0,15,4.053897,0.331154,3.589192,0.247138,"[-0.0076882686, -0.1261441, 0.18335178, -0.115...","[0.00032921784, -0.12477857, 0.18587488, -0.10...",0.998717,"[0.16462258, 0.25113574, 0.38716152, -0.236501...","[-0.65673816, 0.075522676, -0.20840253, 0.3588...",-0.239472
1,4,17681,33728,1.212365e+09,1.0,5.0,15,4.053897,0.331154,4.000000,0.000000,"[-0.0076882686, -0.1261441, 0.18335178, -0.115...","[0.01309923, -0.13805045, 0.20096067, -0.09707...",0.986891,"[0.16462258, 0.25113574, 0.38716152, -0.236501...","[0.35942927, -0.32885966, 0.16308536, 0.031471...",-0.069990
2,10,69,27202,1.353974e+09,1.0,4.0,15,4.053897,0.331154,3.552239,0.403874,"[-0.0076882686, -0.1261441, 0.18335178, -0.115...","[0.0025823559, -0.12812333, 0.17530991, -0.113...",0.998774,"[0.16462258, 0.25113574, 0.38716152, -0.236501...","[0.86368465, 0.048523802, -0.08803976, 0.22050...",0.028235
3,11,69,82543,1.350518e+09,1.0,1.0,15,4.053897,0.331154,3.552239,0.403874,"[-0.0076882686, -0.1261441, 0.18335178, -0.115...","[0.0025823559, -0.12812333, 0.17530991, -0.113...",0.998774,"[0.16462258, 0.25113574, 0.38716152, -0.236501...","[0.86368465, 0.048523802, -0.08803976, 0.22050...",0.028235
4,14,69,18135,1.339200e+09,1.0,3.0,15,4.053897,0.331154,3.552239,0.403874,"[-0.0076882686, -0.1261441, 0.18335178, -0.115...","[0.0025823559, -0.12812333, 0.17530991, -0.113...",0.998774,"[0.16462258, 0.25113574, 0.38716152, -0.236501...","[0.86368465, 0.048523802, -0.08803976, 0.22050...",0.028235


In [45]:
train.drop('idx',axis=1).head()

,index,product_id,user_id,time,helpfulness,score,cate,user_mean_score,user_mean_help,pro_mean_score,pro_mean_help,u_vec,p_vec,dot,u_vec_pca,p_vec_pca,dot_pca
69919,176270,29873,76035,1.200010e+09,0.666667,4.0,57,4.0,0.666667,4.231707,0.274882,"[-0.023424821, -0.11292456, 0.18853657, -0.115...","[-0.021083262, -0.1304503, 0.16853152, -0.1036...",0.996360,"[-0.71635073, 0.12255365, -0.21689054, 0.19962...","[-0.7903225, -0.21802181, 0.05431329, -0.09516...",0.513905
64716,34371,26252,71133,1.200010e+09,1.000000,2.0,15,3.5,0.500000,3.230769,0.461538,"[-0.011791757, -0.12723447, 0.19309308, -0.109...","[0.05222546, -0.12136055, 0.20574287, -0.11758...",0.976426,"[-0.43207568, -0.28160614, -0.37597206, -0.220...","[0.67142487, -0.24425057, -0.3592007, -0.22807...",-0.062924
95170,168595,18051,96666,1.200010e+09,1.000000,5.0,61,5.0,1.000000,4.375000,0.500000,"[-0.02140619, -0.13102354, 0.16955456, -0.1483...","[0.00080324546, -0.12977651, 0.18457516, -0.10...",0.957980,"[-0.23511194, -0.36231145, -0.05822904, 0.3099...","[-0.40747398, 0.28078416, 0.34387022, 0.152414...",0.061640
109872,94854,29346,99204,1.200010e+09,0.857143,4.0,46,4.0,0.857143,4.400000,0.737733,"[-0.03225298, -0.11813351, 0.17550687, -0.1327...","[-0.0184484, -0.13732621, 0.1701287, -0.102108...",0.990932,"[0.5198544, -0.117018275, 0.07950021, 0.314501...","[-0.35075575, 0.14392184, 0.22873686, 0.303211...",0.077728
119840,195123,17331,10901,1.200010e+09,0.000000,5.0,61,5.0,0.500000,5.000000,0.000000,"[-0.027470699, -0.11355752, 0.17480963, -0.135...","[-0.032984387, -0.12086839, 0.17090109, -0.136...",0.998860,"[0.4537118, 0.031590253, 0.70554, -0.15370068,...","[0.51794595, 0.0035934413, 0.5631712, -0.26780...",0.809477


In [46]:
train=train.drop('idx',axis=1)
train=train.drop('helpfulness',axis=1)
test=test.drop('helpfulness',axis=1)
train = train.reindex(columns=[col for col in train.columns if col != 'score'] + ['score'])
test = test.reindex(columns=[col for col in test.columns if col != 'score'] + ['score'])
train.head()

,index,product_id,user_id,time,cate,user_mean_score,user_mean_help,pro_mean_score,pro_mean_help,u_vec,p_vec,dot,u_vec_pca,p_vec_pca,dot_pca,score
69919,176270,29873,76035,1.200010e+09,57,4.0,0.666667,4.231707,0.274882,"[-0.023424821, -0.11292456, 0.18853657, -0.115...","[-0.021083262, -0.1304503, 0.16853152, -0.1036...",0.996360,"[-0.71635073, 0.12255365, -0.21689054, 0.19962...","[-0.7903225, -0.21802181, 0.05431329, -0.09516...",0.513905,4.0
64716,34371,26252,71133,1.200010e+09,15,3.5,0.500000,3.230769,0.461538,"[-0.011791757, -0.12723447, 0.19309308, -0.109...","[0.05222546, -0.12136055, 0.20574287, -0.11758...",0.976426,"[-0.43207568, -0.28160614, -0.37597206, -0.220...","[0.67142487, -0.24425057, -0.3592007, -0.22807...",-0.062924,2.0
95170,168595,18051,96666,1.200010e+09,61,5.0,1.000000,4.375000,0.500000,"[-0.02140619, -0.13102354, 0.16955456, -0.1483...","[0.00080324546, -0.12977651, 0.18457516, -0.10...",0.957980,"[-0.23511194, -0.36231145, -0.05822904, 0.3099...","[-0.40747398, 0.28078416, 0.34387022, 0.152414...",0.061640,5.0
109872,94854,29346,99204,1.200010e+09,46,4.0,0.857143,4.400000,0.737733,"[-0.03225298, -0.11813351, 0.17550687, -0.1327...","[-0.0184484, -0.13732621, 0.1701287, -0.102108...",0.990932,"[0.5198544, -0.117018275, 0.07950021, 0.314501...","[-0.35075575, 0.14392184, 0.22873686, 0.303211...",0.077728,4.0
119840,195123,17331,10901,1.200010e+09,61,5.0,0.500000,5.000000,0.000000,"[-0.027470699, -0.11355752, 0.17480963, -0.135...","[-0.032984387, -0.12086839, 0.17090109, -0.136...",0.998860,"[0.4537118, 0.031590253, 0.70554, -0.15370068,...","[0.51794595, 0.0035934413, 0.5631712, -0.26780...",0.809477,5.0


In [47]:
train['dot_pca'].describe()

count    140495.000000
mean          0.227254
std           0.433456
min          -0.819617
25%          -0.099776
50%           0.159867
75%           0.521226
max           1.000000
Name: dot_pca, dtype: float64

In [48]:
test['dot_pca'].describe()

count    60212.000000
mean        -0.009033
std          0.211534
min         -0.797016
25%         -0.166270
50%         -0.015658
75%          0.140855
max          1.000000
Name: dot_pca, dtype: float64

In [50]:
import pickle 
pickle.dump(train,open('train_1.pkl','wb'))
pickle.dump(test,open('test_1.pkl','wb'))

In [64]:
import pickle 
train=pickle.load(open('train_1.pkl','rb'))
test=pickle.load(open('test_1.pkl','rb'))
train.shape,test.shape

((140495, 16), (60212, 16))

In [65]:
train.head(10)

,index,product_id,user_id,time,cate,user_mean_score,user_mean_help,pro_mean_score,pro_mean_help,u_vec,p_vec,dot,u_vec_pca,p_vec_pca,dot_pca,score
69919,176270,29873,76035,1.200010e+09,57,4.0,0.666667,4.231707,0.274882,"[-0.023424821, -0.11292456, 0.18853657, -0.115...","[-0.021083262, -0.1304503, 0.16853152, -0.1036...",0.996360,"[-0.71635073, 0.12255365, -0.21689054, 0.19962...","[-0.7903225, -0.21802181, 0.05431329, -0.09516...",0.513905,4.0
64716,34371,26252,71133,1.200010e+09,15,3.5,0.500000,3.230769,0.461538,"[-0.011791757, -0.12723447, 0.19309308, -0.109...","[0.05222546, -0.12136055, 0.20574287, -0.11758...",0.976426,"[-0.43207568, -0.28160614, -0.37597206, -0.220...","[0.67142487, -0.24425057, -0.3592007, -0.22807...",-0.062924,2.0
95170,168595,18051,96666,1.200010e+09,61,5.0,1.000000,4.375000,0.500000,"[-0.02140619, -0.13102354, 0.16955456, -0.1483...","[0.00080324546, -0.12977651, 0.18457516, -0.10...",0.957980,"[-0.23511194, -0.36231145, -0.05822904, 0.3099...","[-0.40747398, 0.28078416, 0.34387022, 0.152414...",0.061640,5.0
109872,94854,29346,99204,1.200010e+09,46,4.0,0.857143,4.400000,0.737733,"[-0.03225298, -0.11813351, 0.17550687, -0.1327...","[-0.0184484, -0.13732621, 0.1701287, -0.102108...",0.990932,"[0.5198544, -0.117018275, 0.07950021, 0.314501...","[-0.35075575, 0.14392184, 0.22873686, 0.303211...",0.077728,4.0
119840,195123,17331,10901,1.200010e+09,61,5.0,0.500000,5.000000,0.000000,"[-0.027470699, -0.11355752, 0.17480963, -0.135...","[-0.032984387, -0.12086839, 0.17090109, -0.136...",0.998860,"[0.4537118, 0.031590253, 0.70554, -0.15370068,...","[0.51794595, 0.0035934413, 0.5631712, -0.26780...",0.809477,5.0
49031,157375,12024,111061,1.200010e+09,57,1.0,1.000000,1.000000,1.000000,"[0.07063598, -0.14132941, 0.19438745, -0.07659...","[0.07063598, -0.14132941, 0.19438745, -0.07659...",1.000000,"[0.05758003, -0.18251757, 0.53403085, -0.25567...","[0.05758003, -0.18251757, 0.53403085, -0.25567...",1.000000,1.0
63362,48204,26807,7802,1.200010e+09,37,1.0,1.000000,3.250000,0.500000,"[0.04548262, -0.1708567, 0.17748949, -0.114504...","[-0.0068543144, -0.11628177, 0.18084262, -0.13...",0.970775,"[-0.5835884, 0.03991955, 0.44121817, -0.232013...","[-0.4115487, 0.21072242, -0.0098903105, -0.099...",0.092764,1.0
17755,64525,34376,30096,1.200010e+09,37,1.0,0.947368,2.545455,0.358852,"[0.032015737, -0.11572986, 0.20364352, -0.1004...","[-0.015967058, -0.14222239, 0.05841882, -0.184...",0.675128,"[0.67396325, -0.49716008, 0.073837385, -0.1176...","[-0.535638, 0.20755303, -0.32023048, 0.1212048...",-0.386547,1.0
66143,41219,2716,98701,1.200010e+09,37,4.0,0.000000,4.333333,0.000000,"[0.024428226, -0.12441163, 0.19178839, -0.1443...","[-0.008658467, -0.13227727, 0.18398279, -0.115...",0.991941,"[-0.48562405, -0.15259525, 0.25549433, -0.1457...","[0.6853115, -0.1809827, 0.09894899, 0.17117889...",-0.139848,4.0
34847,17303,14962,15719,1.200010e+09,15,5.0,0.000000,4.000000,0.200000,"[-0.0016687587, -0.13036276, 0.18535446, -0.12...","[0.014951322, -0.10439513, 0.20247015, -0.1011...",0.976181,"[0.28643173, -0.32515657, -0.41506505, -0.2557...","[-0.24146016, -0.290135, -0.29979873, 0.127732...",0.370579,5.0


In [66]:
test.head(10)

,index,product_id,user_id,time,cate,user_mean_score,user_mean_help,pro_mean_score,pro_mean_help,u_vec,p_vec,dot,u_vec_pca,p_vec_pca,dot_pca,score
0,0,30088,34501,1.233360e+09,15,4.053897,0.331154,3.589192,0.247138,"[-0.0076882686, -0.1261441, 0.18335178, -0.115...","[0.00032921784, -0.12477857, 0.18587488, -0.10...",0.998717,"[0.16462258, 0.25113574, 0.38716152, -0.236501...","[-0.65673816, 0.075522676, -0.20840253, 0.3588...",-0.239472,1.0
1,4,17681,33728,1.212365e+09,15,4.053897,0.331154,4.000000,0.000000,"[-0.0076882686, -0.1261441, 0.18335178, -0.115...","[0.01309923, -0.13805045, 0.20096067, -0.09707...",0.986891,"[0.16462258, 0.25113574, 0.38716152, -0.236501...","[0.35942927, -0.32885966, 0.16308536, 0.031471...",-0.069990,5.0
2,10,69,27202,1.353974e+09,15,4.053897,0.331154,3.552239,0.403874,"[-0.0076882686, -0.1261441, 0.18335178, -0.115...","[0.0025823559, -0.12812333, 0.17530991, -0.113...",0.998774,"[0.16462258, 0.25113574, 0.38716152, -0.236501...","[0.86368465, 0.048523802, -0.08803976, 0.22050...",0.028235,4.0
3,11,69,82543,1.350518e+09,15,4.053897,0.331154,3.552239,0.403874,"[-0.0076882686, -0.1261441, 0.18335178, -0.115...","[0.0025823559, -0.12812333, 0.17530991, -0.113...",0.998774,"[0.16462258, 0.25113574, 0.38716152, -0.236501...","[0.86368465, 0.048523802, -0.08803976, 0.22050...",0.028235,1.0
4,14,69,18135,1.339200e+09,15,4.053897,0.331154,3.552239,0.403874,"[-0.0076882686, -0.1261441, 0.18335178, -0.115...","[0.0025823559, -0.12812333, 0.17530991, -0.113...",0.998774,"[0.16462258, 0.25113574, 0.38716152, -0.236501...","[0.86368465, 0.048523802, -0.08803976, 0.22050...",0.028235,3.0
5,15,69,110360,1.330733e+09,15,4.053897,0.331154,3.552239,0.403874,"[-0.0076882686, -0.1261441, 0.18335178, -0.115...","[0.0025823559, -0.12812333, 0.17530991, -0.113...",0.998774,"[0.16462258, 0.25113574, 0.38716152, -0.236501...","[0.86368465, 0.048523802, -0.08803976, 0.22050...",0.028235,4.0
6,16,69,86608,1.330128e+09,15,4.053897,0.331154,3.552239,0.403874,"[-0.0076882686, -0.1261441, 0.18335178, -0.115...","[0.0025823559, -0.12812333, 0.17530991, -0.113...",0.998774,"[0.16462258, 0.25113574, 0.38716152, -0.236501...","[0.86368465, 0.048523802, -0.08803976, 0.22050...",0.028235,5.0
7,18,69,142945,1.326758e+09,15,4.053897,0.331154,3.552239,0.403874,"[-0.0076882686, -0.1261441, 0.18335178, -0.115...","[0.0025823559, -0.12812333, 0.17530991, -0.113...",0.998774,"[0.16462258, 0.25113574, 0.38716152, -0.236501...","[0.86368465, 0.048523802, -0.08803976, 0.22050...",0.028235,5.0
8,19,69,135296,1.309997e+09,15,4.053897,0.331154,3.552239,0.403874,"[-0.0076882686, -0.1261441, 0.18335178, -0.115...","[0.0025823559, -0.12812333, 0.17530991, -0.113...",0.998774,"[0.16462258, 0.25113574, 0.38716152, -0.236501...","[0.86368465, 0.048523802, -0.08803976, 0.22050...",0.028235,5.0
9,20,69,116602,1.307923e+09,15,4.053897,0.331154,3.552239,0.403874,"[-0.0076882686, -0.1261441, 0.18335178, -0.115...","[0.0025823559, -0.12812333, 0.17530991, -0.113...",0.998774,"[0.16462258, 0.25113574, 0.38716152, -0.236501...","[0.86368465, 0.048523802, -0.08803976, 0.22050...",0.028235,5.0


In [59]:
import pandas as pd
df=pd.concat([train,test])
df['dot'].corr(df['score']),df['dot_pca'].corr(df['score'])

(0.1400054702541495, -0.01569310780861725)

In [60]:
from scipy.stats import pearsonr
pearsonr(df['dot'], df['score'])

PearsonRResult(statistic=0.14000547025414964, pvalue=0.0)

In [68]:
df['user_id'].max(),df['user_id'].nunique(),df['product_id'].max(),df['product_id'].nunique()

(165532, 165533, 37511, 37512)

In [69]:
df.isnull().any()

index              False
product_id         False
user_id            False
time               False
cate               False
user_mean_score    False
user_mean_help     False
pro_mean_score     False
pro_mean_help      False
u_vec              False
p_vec              False
dot                False
u_vec_pca          False
p_vec_pca          False
dot_pca            False
score              False
dtype: bool

In [63]:
df['index'].nunique()

200707

In [67]:
train.sort_index().head()

,index,product_id,user_id,time,cate,user_mean_score,user_mean_help,pro_mean_score,pro_mean_help,u_vec,p_vec,dot,u_vec_pca,p_vec_pca,dot_pca,score
0,145565,20310,104238,1.284768e+09,46,5.0,0.0,5.000000,0.000000,"[-0.02346941, -0.09834979, 0.19334072, -0.1018...","[-0.02346941, -0.09834979, 0.19334072, -0.1018...",1.000000,"[0.05889, 0.06314615, 0.12501675, -0.033957116...","[0.05889, 0.06314615, 0.12501675, -0.033957116...",1.000000,5.0
1,63962,23917,135920,1.289866e+09,37,5.0,0.0,3.887324,0.278946,"[-0.026852, -0.115601875, 0.17347999, -0.12015...","[-0.009399407, -0.14125589, 0.16808625, -0.100...",0.990067,"[0.5394346, 0.042229325, 0.05724875, -0.140100...","[-0.503824, -0.41808432, 0.26737812, 0.2120547...",-0.143354,5.0
2,33012,31974,98890,1.250467e+09,15,3.0,0.0,2.818182,0.363636,"[-0.004658646, -0.118791305, 0.18418999, -0.09...","[0.012678218, -0.11893866, 0.18927261, -0.1053...",0.994303,"[-0.42745692, -0.07449984, -0.17510243, 0.5122...","[-0.37867337, -0.14831817, 0.17387894, 0.08032...",0.259903,3.0
3,44561,33923,3566,1.282781e+09,37,5.0,0.0,4.285714,0.000000,"[0.008941505, -0.137913, 0.18020944, -0.116109...","[-0.0034881837, -0.12515078, 0.18736587, -0.11...",0.996295,"[0.64223784, -0.060188215, -0.35695258, -0.203...","[-0.38090807, -0.15082082, 0.049121287, -0.189...",-0.196117,5.0
4,92663,13685,4502,1.271635e+09,46,2.0,0.0,2.923077,0.497894,"[0.019391282, -0.113317184, 0.21154177, -0.087...","[-0.019406183, -0.13300925, 0.1826488, -0.1119...",0.975654,"[-0.2962679, -0.15704587, 0.083281055, 0.11136...","[0.54213876, -0.007821561, 0.643091, -0.013266...",-0.072780,2.0


In [71]:
test.head()

,index,product_id,user_id,time,cate,user_mean_score,user_mean_help,pro_mean_score,pro_mean_help,u_vec,p_vec,dot,u_vec_pca,p_vec_pca,dot_pca,score
0,0,30088,34501,1.233360e+09,15,4.053897,0.331154,3.589192,0.247138,"[-0.0076882686, -0.1261441, 0.18335178, -0.115...","[0.00032921784, -0.12477857, 0.18587488, -0.10...",0.998717,"[0.16462258, 0.25113574, 0.38716152, -0.236501...","[-0.65673816, 0.075522676, -0.20840253, 0.3588...",-0.239472,1.0
1,4,17681,33728,1.212365e+09,15,4.053897,0.331154,4.000000,0.000000,"[-0.0076882686, -0.1261441, 0.18335178, -0.115...","[0.01309923, -0.13805045, 0.20096067, -0.09707...",0.986891,"[0.16462258, 0.25113574, 0.38716152, -0.236501...","[0.35942927, -0.32885966, 0.16308536, 0.031471...",-0.069990,5.0
2,10,69,27202,1.353974e+09,15,4.053897,0.331154,3.552239,0.403874,"[-0.0076882686, -0.1261441, 0.18335178, -0.115...","[0.0025823559, -0.12812333, 0.17530991, -0.113...",0.998774,"[0.16462258, 0.25113574, 0.38716152, -0.236501...","[0.86368465, 0.048523802, -0.08803976, 0.22050...",0.028235,4.0
3,11,69,82543,1.350518e+09,15,4.053897,0.331154,3.552239,0.403874,"[-0.0076882686, -0.1261441, 0.18335178, -0.115...","[0.0025823559, -0.12812333, 0.17530991, -0.113...",0.998774,"[0.16462258, 0.25113574, 0.38716152, -0.236501...","[0.86368465, 0.048523802, -0.08803976, 0.22050...",0.028235,1.0
4,14,69,18135,1.339200e+09,15,4.053897,0.331154,3.552239,0.403874,"[-0.0076882686, -0.1261441, 0.18335178, -0.115...","[0.0025823559, -0.12812333, 0.17530991, -0.113...",0.998774,"[0.16462258, 0.25113574, 0.38716152, -0.236501...","[0.86368465, 0.048523802, -0.08803976, 0.22050...",0.028235,3.0


In [74]:
tmp=test.groupby(['user_id'])['index'].nunique().reset_index()
tmp['index'].describe()

count    54990.000000
mean         1.094963
std          0.643381
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max         97.000000
Name: index, dtype: float64

In [76]:
train['user_id'].nunique()

120471